In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, BartForConditionalGeneration, AutoModelForSeq2SeqLM

from datasets import load_dataset
from huggingface_hub import login

/home2/likhithasapu/miniconda3/envs/research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prakod/gcm_codemix_indicbart_cmi_ratings_gt_4_cmi_gt_30_allperidx")
model = AutoModelForSeq2SeqLM.from_pretrained("prakod/gcm_codemix_indicbart_cmi_ratings_gt_4_cmi_gt_30_allperidx").to("cuda")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home2/likhithasapu/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
text = "What do you call a cow wearing sunglasses?"
input_text = f"Translate the English sentence to Hindi-English sentence: <s> {text} </s>"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids = input_ids["input_ids"], attention_mask = input_ids["attention_mask"], max_new_tokens=150, num_beams=5, no_repeat_ngram_size=3, temperature=1, do_sample=True, top_k=50, top_p=0.95)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

What do you call a cow wearing sunglasses?


In [13]:
import json
from tqdm import tqdm
import re
# Read data from JSON file
with open('task2/Chatgpt_pun_translated.json', 'r') as file:
    data = json.load(file)

phrase_pattern = r"Generated Small Phrase:\s*(.*)"
pun_word_pattern = r"Replace Pun Word:\s*(.*)"
question_pattern = r"Generated Question:\s*(.*)"

def extract_question_and_answer(text):
    # Extract using regex
    generated_phrase = re.search(phrase_pattern, text).group(1)
    replace_pun_word = re.search(pun_word_pattern, text).group(1)
    generated_question = re.search(question_pattern, text).group(1)
    
    return generated_question, replace_pun_word
    
# Translate the "pun" field and add a new field "pun_translated"
for item in tqdm(data,total=len(data)):
    text = item['Response']
    question, answer = extract_question_and_answer(text)
    input_text = f"Translate the English sentence to Hindi-English sentence: <s> {question} </s>"
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=input_ids["input_ids"], attention_mask=input_ids["attention_mask"], max_new_tokens=150, num_beams=5, no_repeat_ngram_size=2, temperature=2.0, do_sample=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    item['Sentence_chosen_translated'] = translated_text + "\n" + answer

# Save the updated data to JSON file
with open('task2/Chatgpt_pun_translated.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

100%|██████████| 20/20 [00:04<00:00,  4.98it/s]
